1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
   - 토큰 수 초과로 답변을 생성하지 못할 수 있음
   - 문서가 길면 (input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [ ]:
# DOCX 파일을 로드하기 위한 로더 생성
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# tax.docx 파일을 읽을 로더 객체 생성
loader = Docx2txtLoader("./tax_with_markdown.docx")

# 문서 로드 (Document 객체 리스트 반환)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()  # .env 파일에서 환경 변수 로드

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-markdown-index"

database = PineconeVectorStore.from_documents(
    document_list,
    embedding,
    index_name=index_name,
)

In [41]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# 유사도 검색 테스트
retrieved_docs = database.similarity_search(query, k=5)

In [42]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [51]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

# 질문 변환 프롬프트
question_transform_prompt = ChatPromptTemplate.from_template(
    f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    
    [사전]
    : {dictionary}
    
    [사용자의 질문]
    : {{question}}
    """
)

# RAG 답변 생성 프롬프트
rag_prompt = ChatPromptTemplate.from_template(
    """
    [Identity]
    - 당신은 최고의 한국 소득세 전문가입니다.
    - [Context]를 참고해서 사용자의 질문에 답변해주세요.

    [Context]
    {context}

    [Question]
    {question}
    """
)

dictionary_chain = question_transform_prompt | llm | StrOutputParser()

In [ ]:
# Retriever 생성
retriever = database.as_retriever(search_kwargs={"k": 2})

# 전체 체인: 질문 변환 -> 검색 -> RAG 답변 생성
rag_chain = (
    {
        "context": dictionary_chain | retriever,
        "question": dictionary_chain,
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [55]:
ai_message = rag_chain.invoke(query)

ai_message

'연봉 5천만원인 거주자의 소득세를 계산하기 위해 해당 과세표준에 적용되는 세율을 살펴보겠습니다.\n\n1. **연봉**: 5,000만원\n2. **과세표준**: 5,000만원\n\n**소득세 계산:**\n\n- **1,400만원 이하**: 1,400만원 × 6% = 84만원\n- **1,400만원 초과 5,000만원 이하**: (5,000만원 - 1,400만원) × 15% + 84만원 = (3,600만원) × 15% + 84만원\n\n계산해보면:\n- 3,600만원 × 15% = 540만원\n- 따라서, 총 소득세 = 540만원 + 84만원 = 624만원\n\n따라서, 연봉 5천만원인 거주자의 소득세는 **624만원**입니다.'